<a href="https://colab.research.google.com/github/Hongchenglong/colab/blob/main/CertifiableBayesianInference/FCN_Experiments/MNIST_runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 装载Google硬盘
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys, os
from pathlib import Path
path = Path(os.getcwd())
sys.path.append(str(path.parent))
sys.path.append('/content/drive/MyDrive/ColabNotebooks/CertifiableBayesianInference')
experiment = '/content/drive/MyDrive/ColabNotebooks/CertifiableBayesianInference/FCN_Experiments/'

In [3]:
# python3 MNIST_runner.py --eps 0.11 --lam 0.25 --rob 0 --opt HMC --gpu 0 &
# python3 MNIST_runner.py --eps 0.11 --lam 0.25 --rob 2 --opt HMC --gpu 1 &
# python3 MNIST_runner.py --eps 0.11 --lam 0.25 --rob 2 --opt BBB --gpu 1 &


# dict = {'eps': 0.11, 'lam': 0.25, 'rob': 0, 'opt': 'HMC', 'gpu': '0'}
# dict = {'eps': 0.11, 'lam': 0.25, 'rob': 2, 'opt': 'HMC', 'gpu': '1'}
dict = {'eps': 0.11, 'lam': 0.25, 'rob': 1, 'opt': 'HMC', 'gpu': '1'}


# epsilon:  the strength of the adversary, 在测试点周围多大的区域里考虑它的鲁棒性
eps = dict['eps']
# lamada: the probability of seeing the clean data (lambda = 1.0 means no adversarial data considered, lambda = 0.0 means only adversarial data considered)
lam = dict['lam'] 
# robustness mode, 0:标准 1:IBP 2:PGD
rob = dict['rob'] 
# the variational inference method
optim = dict['opt']
gpu = dict['gpu']
os.environ['CUDA_VISIBLE_DEVICES'] = gpu

In [4]:
# 作者封装的BayesKeras模块
import BayesKeras
import BayesKeras.optimizers as optimizers

import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
# 划分训练集，测试集
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train/255.
X_test = X_test/255.
X_train = X_train.astype("float32").reshape(-1, 28*28)
X_test = X_test.astype("float32").reshape(-1, 28*28)

#X_train = X_train[0:10000]
#y_train = y_train[0:10000]

# 构建Sequential模型
model = Sequential()
# 构建Dense隐藏层并添加到模型中
# 添加具有512个神经元的Dense隐藏层，使用relu激活函数
model.add(Dense(512, activation="relu", input_shape=(None, None, 28*28)))
# 添加具有10个神经元的Dense隐藏层，使用softmax激活函数
model.add(Dense(10, activation="softmax"))

inf = 2
full_covar = False
if(optim == 'VOGN'):
    # was 0.25 for a bit
    inf = 2
    learning_rate = 0.35; decay=0.0
    opt = optimizers.VariationalOnlineGuassNewton()
elif(optim == 'BBB'):
    inf = 10
    learning_rate = 0.45; decay=0.0
    opt = optimizers.BayesByBackprop()
elif(optim == 'SWAG'):
    learning_rate = 0.01; decay=0.0
    opt = optimizers.StochasticWeightAveragingGaussian()
elif(optim == 'SWAG-FC'):
    learning_rate = 0.01; decay=0.0; full_covar=True
    # 优化器.随机加权平均高斯
    opt = optimizers.StochasticWeightAveragingGaussian()
elif(optim == 'SGD'):
    learning_rate = 1.0; decay=0.0
    opt = optimizers.StochasticGradientDescent()
elif(optim == 'NA'):
    inf = 2
    learning_rate = 0.001; decay=0.0
    opt = optimizers.NoisyAdam()
elif(optim == 'ADAM'):
    learning_rate = 0.00001; decay=0.0
    opt = optimizers.Adam()
elif(optim == 'HMC'):
    learning_rate = 0.075; decay=0.0; inf=250
    linear_schedule = False
    opt = optimizers.HamiltonianMonteCarlo()

# Compile the model to train with Bayesian inference
if(rob == 0):
    # 稀疏分类交叉熵
    loss = tf.keras.losses.SparseCategoricalCrossentropy()
elif(rob != 0):
    # 鲁棒交叉熵损失函数
    loss = BayesKeras.optimizers.losses.robust_crossentropy_loss

bayes_model = opt.compile(model, loss_fn=loss, epochs=20, learning_rate=learning_rate,
                          batch_size=128, linear_schedule=True,
                          decay=decay, robust_train=rob, inflate_prior=inf,
                          burn_in=3, steps=25, b_steps=20, epsilon=eps, rob_lam=lam) #, preload="SGD_FCN_Posterior_1")

11501568/11490434 [==============================] - 2s 0us/step
This optimizer does not have a default compilation method. Please make sure to call the correct .compile method before use.
BayesKeras: Using implicit prior
(784, 512) 0.5646924393157821
(512, 10) 0.6987712429686843
BayesKeras: Using implicit prior
(784, 512) 0.5646924393157821
(512, 10) 0.6987712429686843
BayesKeras: Detected robust training at compilation. Please ensure you have selected a robust-compatible loss
<class 'list'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
[<tf.Tensor: shape=(784, 512), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0.

In [6]:
# steps was 50
# Train the model on your data
bayes_model.train(X_train, y_train, X_test, y_test)

BayesKeras Warning: HMC is a memory hungry optimizer. 
         Given you system and parameters of this training run,
         we expect your system to need 0.002496 MB of available memory


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
Numerical Integration: 100%|██████████| 20/20 [01:39<00:00,  5.00s/it]


Current  U:  tf.Tensor(4.148409, shape=(), dtype=float32)
Proposed U:  tf.Tensor(2.5728998, shape=(), dtype=float32)
Current  K:  tf.Tensor(0.008359623, shape=(), dtype=float32)
Proposed K:  tf.Tensor(0.017667681, shape=(), dtype=float32)
METROPOLIS CORRECTION CONSTANT:  tf.Tensor(4.7884226, shape=(), dtype=float32)
ACCEPTED
Debug info:
[0, 1]
Epoch 1, loss: 3.125, acc: 0.619, val_loss: 0.615, val_acc: 0.877, rob: 0.526, (eps = 0.000000)
Still in Burn-In state of Markov chain.


Numerical Integration: 100%|██████████| 20/20 [01:40<00:00,  5.02s/it]


Current  U:  tf.Tensor(2.5728998, shape=(), dtype=float32)
Proposed U:  tf.Tensor(3.8568068, shape=(), dtype=float32)
Current  K:  tf.Tensor(0.0067873737, shape=(), dtype=float32)
Proposed K:  tf.Tensor(0.0073204683, shape=(), dtype=float32)
METROPOLIS CORRECTION CONSTANT:  tf.Tensor(0.27680555, shape=(), dtype=float32)
ACCEPTED
Debug info:
[0, 1]
Epoch 2, loss: 3.870, acc: 0.876, val_loss: 1.876, val_acc: 0.867, rob: 0.000, (eps = 0.036667)
Still in Burn-In state of Markov chain.


...

Current  U:  tf.Tensor(5.8237495, shape=(), dtype=float32)
Proposed U:  tf.Tensor(5.9327393, shape=(), dtype=float32)
Current  K:  tf.Tensor(0.014620271, shape=(), dtype=float32)
Proposed K:  tf.Tensor(0.014657633, shape=(), dtype=float32)
METROPOLIS CORRECTION CONSTANT:  tf.Tensor(0.89670616, shape=(), dtype=float32)
ACCEPTED
Debug info:
[0, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1]
Epoch 19, loss: 5.865, acc: 0.921, val_loss: 2.186, val_acc: 0.917, rob: 0.000, (eps = 0.110000)
Out of Burn-In state. Generating samples from the chain


Numerical Integration: 100%|██████████| 25/25 [02:04<00:00,  4.96s/it]


Current  U:  tf.Tensor(5.9327393, shape=(), dtype=float32)
Proposed U:  tf.Tensor(5.964735, shape=(), dtype=float32)
Current  K:  tf.Tensor(0.018864272, shape=(), dtype=float32)
Proposed K:  tf.Tensor(0.018780831, shape=(), dtype=float32)
METROPOLIS CORRECTION CONSTANT:  tf.Tensor(0.9685915, shape=(), dtype=float32)
ACCEPTED
Debug info:
[0, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1]
Epoch 20, loss: 5.930, acc: 0.920, val_loss: 2.237, val_acc: 0.915, rob: 0.000, (eps = 0.110000)
Out of Burn-In state. Generating samples from the chain


Numerical Integration: 100%|██████████| 25/25 [02:04<00:00,  4.98s/it]


Current  U:  tf.Tensor(5.964735, shape=(), dtype=float32)
Proposed U:  tf.Tensor(6.0421104, shape=(), dtype=float32)
Current  K:  tf.Tensor(0.014798769, shape=(), dtype=float32)
Proposed K:  tf.Tensor(0.014863508, shape=(), dtype=float32)
METROPOLIS CORRECTION CONSTANT:  tf.Tensor(0.9254824, shape=(), dtype=float32)
ACCEPTED
Debug info:
[0, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1]
Epoch 21, loss: 5.991, acc: 0.919, val_loss: 2.244, val_acc: 0.917, rob: 0.000, (eps = 0.110000)
Out of Burn-In state. Generating samples from the chain


In [7]:
# Save your approxiate Bayesian posterior
bayes_model.save(experiment + "%s_FCN_Posterior_%s"%(optim, rob))

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
